In [142]:
import numpy as np
import pandas as pd
import cStringIO
import psycopg2 as pg

pd.set_option("MAX_ROWS", 100)

In [99]:
def create_grid_coords(nrows, ncols, xllcenter, yllcenter, cellsize):
    """Create a vector of gridded coordinates.
        
    returns: (x , y , coords) where
        x is an RxC ndarray of xcoords for each cimis grid centroid
        y is an RxC ndarray of ycoords for each cimis grid centroid
        coords is an RxC chararray of unique identifiers of the form for 'XxYy' (e.g. '-409000x459000y')
            with X representing the xcoord of that grid cell and Y the ycoord of the grid cell
    """
    # create matrix of x coordinates for cells in EPSG 3310
    x = np.ndarray((nrows,ncols)) + xllcenter
    offsets = cellsize*np.arange(ncols)
    x = x + offsets
    
    # create matrix of y coordinates for cells in EPSG 3310
    y = np.ndarray((nrows,ncols)) + yllcenter
    offsets = cellsize*np.arange(nrows)
    y = np.flipud(np.transpose(np.transpose(y) + offsets))
    
    # create matrix of unique identifiers for each cell by appending x and y coords into a string
    xcoords = np.core.defchararray.add(x.astype(int).astype(str), "x" )
    ycoords = np.core.defchararray.add(y.astype(int).astype(str), "y" )
    coords = np.core.defchararray.add(xcoords, ycoords)
    
    return (x, y, coords)

In [101]:
def get_formatted_url(year, month, day, data_field):
    """Create a formatted url to query for a single Spatial CIMIS data file.
    
    returns: url a formatted url string
    """
    url = "http://cimis.casil.ucdavis.edu/cimis/{}/{:02d}/{:02d}/{}.asc.gz".format(year, month, day, data_field)
    return url

In [116]:
from StringIO import StringIO
import gzip
import urllib2

request = urllib2.Request("http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Rso.asc.gz")
request.add_header('Accept-encoding', 'gzip')
response = urllib2.urlopen(request)

In [117]:
if response.info().get('Content-Encoding') == 'gzip':
    buf = StringIO(response.read())
    f = gzip.GzipFile(fileobj=buf)

In [134]:
f.readline()

'NODATA_value -9999\n'

In [167]:
rso_coord_vec = create_grid_coords(552, 500, -399000, -649000, 2000)

In [132]:
def parse_spatial_cimis_field(url, vec_len):
    """Fetch Spatial CIMIS data in asc format from the url and parse into a 'long' output vector.
    
    return: CIMIS data reshaped into a single pandas Series (row-major format)
    """
    #df = pd.read_csv("../../../../Dropbox/CIMIS/ET2003_02_20.asc", sep=" ", header=None, skiprows=6)
    df = pd.read_csv(url, compression="gzip", sep=" ", header=None, skiprows=6)
    df = df.loc[:,df.mean().notnull()]
    print df.shape
    return df.values.reshape(vec_len)

In [86]:
def get_tidy_cimis_daily(year, month, day, coord_vec=None):
    """Query all of the different Spatial CIMIS data fields for a single day and
    assemble them into a dataframe format suitable for storing in a relational database. 
    This includes the addition of contextual fields like coordinates, date fields,
    and a unique identifier.
    
    returns: output a pandas dataframe where each row holds all of the measurements for a given grid cell
        on a particular date.
    """
    
    print "Parsing data for", pd.datetime(year, month, day)
    
    if coord_vec is None:
        coord_vec = create_grid_coords()
        
    x, y, coords = coord_vec
    
    out_dict = {
        "sc_cell_id": coords.reshape(total_cells),
        "sc_x": x.reshape(total_cells).astype(int),
        "sc_y": y.reshape(total_cells).astype(int),
        "sc_year": year,
        "sc_month": month,
        "sc_day": day,
        "sc_date": pd.datetime(year, month, day)
    }
    
    fieldlist = ["ETo", "K", "Rnl", "Rs", #"Rso", "Tdew", "Tn", "Tx", "U2"]
    
    for field in fieldlist:
        url = get_formatted_url(year, month, day, field)
        print url
        fieldname = "sc_{}".format(field).lower()
        out_dict[fieldname] = parse_spatial_cimis_field(url)
                          
    
    # assemble output dataframe by melting/reshaping above matrices into single columns
    output = pd.DataFrame(out_dict).replace(NODATA_value, np.nan)
    output = output.dropna(how="all", subset=fieldlist)
    
    return output

In [144]:
def get_tidy_cimis_daily(year, month, day, coord_vec=None, rso_coord_vec=None):
    """Query all of the different Spatial CIMIS data fields for a single day and
    assemble them into a dataframe format suitable for storing in a relational database. 
    This includes the addition of contextual fields like coordinates, date fields,
    and a unique identifier.
    
    returns: output a pandas dataframe where each row holds all of the measurements for a given grid cell
        on a particular date.
    """
    if coord_vec is None:
        coord_vec = create_grid_coords(nrows, ncols, xllcorner, yllcorner, cellsize)
    if rso_coord_vec is None:
        rso_coord_vec = create_grid_coords(552, 500, -399000, -649000, 2000)

    #get data for all filds except Rso
    x, y, coords = coord_vec
    out_dict = {
        "sc_cell_id": coords.reshape(total_cells),
        "sc_x": x.reshape(total_cells).astype(int),
        "sc_y": y.reshape(total_cells).astype(int),
        "sc_year": year,
        "sc_month": month,
        "sc_day": day,
        "sc_date": pd.datetime(year, month, day)
    }

    fieldlist = ["ETo", "K", "Rnl", "Rs", "Tdew", "Tn", "Tx", "U2"]

    for field in fieldlist:
        url = get_formatted_url(year, month, day, field)
        print url
        fieldname = "sc_{}".format(field).lower()
        out_dict[fieldname] = parse_spatial_cimis_field(url, total_cells)


    #special treatment for Rso because of different matrix size
    x, y, rso_coords = rso_coord_vec
    rso_out_dict = {
        "sc_cell_id": rso_coords.reshape(552*500),
    }

    field = "Rso"
    url = get_formatted_url(year, month, day, field)
    print url
    fieldname = "sc_{}".format(field).lower()
    rso_out_dict[fieldname] = parse_spatial_cimis_field(url, 552*500)

    #merge outputs into a single dataframe and drop nulls
    output = pd.DataFrame(out_dict).replace(NODATA_value, np.nan)
    rso_output = pd.DataFrame(rso_out_dict).replace(NODATA_value, np.nan)
    output = output.merge(rso_output, how="left", on="sc_cell_id")
    output = output.dropna(how="all", subset=['sc_eto'])
    
    return output

http://cimis.casil.ucdavis.edu/cimis/2003/02/20/ETo.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/K.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Rnl.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Rs.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Tdew.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Tn.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/Tx.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2003/02/20/U2.asc.gz
(560, 510)


In [87]:
def get_tidy_cimis_daterange(start_date=None, end_date=None, coord_vec=None):

    if start_date < pd.datetime("2003-02-20"):
        raise ValueError("no data available before 2003-02-20")
    
    if start_date > end_date:
        raise ValueError("start_date must be <= end_date")
    
    print "Fetching data from", start_date, "to", end_date
    
    if coord_vec is None:
        coord_vec = create_grid_coords()
        
    #TODO iterate over the dates in the range and call tidy_cimis_daily for each

In [88]:
def to_pg(df, tbl_name, conn, cols):
    output = cStringIO.StringIO()
    df.to_csv(output, sep="\t", header=False, index=False)
    output.getvalue()
    output.seek(0)
    
    cur = conn.cursor()
    cur.copy_from(output, tbl_name, null="", columns=(cols))
    conn.commit()
    cur.close()

In [89]:
ncols = 510
nrows = 560
xllcorner = -410000
yllcorner = -660000
cellsize = 2000.000000
NODATA_value = -9999

total_cells = nrows*ncols
xllcenter = xllcorner + int(cellsize/2)
yllcenter = yllcorner + int(cellsize/2)

coord_vec = create_grid_coords()

year = 2003
month = 2
day = 20



In [90]:
df = get_tidy_cimis_daily(2004, 2, 20, coord_vec)

Parsing data for 2004-02-20 00:00:00
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/ETo.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/K.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/Rnl.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/Rs.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/Tdew.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/Tn.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/Tx.asc.gz
(560, 510)
http://cimis.casil.ucdavis.edu/cimis/2004/02/20/U2.asc.gz
(560, 510)


KeyError: ['ETo', 'K', 'Rnl', 'Rs', 'Tdew', 'Tn', 'Tx', 'U2']

In [198]:
dbname = 'postgres'
dbuser = 'postgres'
dbpass = '*****'
dbhost = 'localhost'
dbport = 5433

In [35]:
connection = pg.connect(database=dbname,
                                user=dbuser,
                                password=dbpass,
                                host=dbhost,
                                port=dbport)

to_pg(df, "spatial_cimis_reading", connection, cols=df.columns.values)

In [21]:
df.head()

,sc_cell_id,sc_date,sc_day,sc_eto,sc_k,sc_month,sc_rnl,sc_rs,sc_rso,sc_tdew,sc_tn,sc_tx,sc_u2,sc_x,sc_y,sc_year
0,-409000x459000y,2003-02-20,20,-9999.0,-9999.0,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-409000.0,459000.0,2003
1,-407000x459000y,2003-02-20,20,-9999.0,-9999.0,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-407000.0,459000.0,2003
2,-405000x459000y,2003-02-20,20,-9999.0,-9999.0,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-405000.0,459000.0,2003
3,-403000x459000y,2003-02-20,20,-9999.0,-9999.0,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-403000.0,459000.0,2003
4,-401000x459000y,2003-02-20,20,-9999.0,-9999.0,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-401000.0,459000.0,2003
